In [29]:
import wrds
import pandas as pd
import numpy as np
import tqdm
# r2r*+pSr/!YhaW.
# Connect to WRDS
db = wrds.Connection()
# Query the WRDS TAQ database


WRDS recommends setting up a .pgpass file.
Created .pgpass file successfully.
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done


### SP500 Definition

In [36]:
sp500 = db.raw_sql("""
                        select a.*, b.date, b.ret
                        from crsp.dsp500list as a,
                        crsp.dsf as b
                        where a.permno=b.permno
                        and b.date >= a.start and b.date<= a.ending
                        and b.date>='01/01/2004' and b.date <= '12/31/2020'
                        order by date;
                        """, date_cols=['start', 'ending', 'date']) 

### MSE (monthly stock events)

In [37]:
mse = db.raw_sql("""
                        select comnam, cusip, ncusip, namedt, nameendt, 
                        permno, shrcd, exchcd, hsiccd, ticker, tsymbol
                        from crsp.msenames
                        """, date_cols=['namedt', 'nameendt'])
mse

,comnam,cusip,ncusip,namedt,nameendt,permno,shrcd,exchcd,hsiccd,ticker,tsymbol
0,OPTIMUM MANUFACTURING INC,68391610,68391610,1986-01-07,1986-12-03,10000,10,3,3990,OMFGA,OMFGA
1,OPTIMUM MANUFACTURING INC,68391610,68391610,1986-12-04,1987-03-09,10000,10,3,3990,OMFGA,OMFAC
2,OPTIMUM MANUFACTURING INC,68391610,68391610,1987-03-10,1987-06-11,10000,10,3,3990,OMFGA,OMFGA
3,GREAT FALLS GAS CO,36720410,39040610,1986-01-09,1993-11-21,10001,11,3,4925,GFGC,GFGC
4,ENERGY WEST INC,36720410,29274A10,1993-11-22,2004-06-09,10001,11,3,4925,EWST,EWST
...,...,...,...,...,...,...,...,...,...,...,...
117825,SINO CLEAN ENERGY INC,82936G20,82936G20,2010-06-14,2012-05-18,93435,11,3,6163,SCEI,SCEI
117826,TESLA MOTORS INC,88160R10,88160R10,2010-06-29,2017-02-01,93436,11,3,9999,TSLA,TSLA
117827,TESLA INC,88160R10,88160R10,2017-02-02,2023-04-02,93436,11,3,9999,TSLA,TSLA
117828,TESLA INC,88160R10,88160R10,2023-04-03,2024-06-19,93436,11,3,9999,TSLA,TSLA


### Merge 

In [38]:
sp500_full = pd.merge(sp500, mse, how = 'left', on = 'permno')

sp500_full = sp500_full.loc[(sp500_full.date>=sp500_full.namedt) \
                            & (sp500_full.date<=sp500_full.nameendt)]



In [41]:
ccm=db.raw_sql("""
                  select gvkey, liid as iid, lpermno as permno, linktype, linkprim, 
                  linkdt, linkenddt
                  from crsp.ccmxpf_linktable
                  where substr(linktype,1,1)='L'
                  and (linkprim ='C' or linkprim='P')
                  """, date_cols=['linkdt', 'linkenddt'])

ccm['linkenddt']=ccm['linkenddt'].fillna(pd.to_datetime('today'))

In [42]:
sp500ccm = pd.merge(sp500_full, ccm, how='left', on=['permno'])

sp500ccm = sp500ccm.loc[(sp500ccm['date']>=sp500ccm['linkdt'])\
                        &(sp500ccm['date']<=sp500ccm['linkenddt'])]

In [43]:
sp500ccm = sp500ccm.drop(columns=['namedt', 'nameendt', \
                                  'linktype', 'linkprim', 'linkdt', 'linkenddt'])
sp500ccm

,permno,start,ending,date,ret,comnam,cusip,ncusip,shrcd,exchcd,hsiccd,ticker,tsymbol,gvkey,iid
0,10078,1992-08-20,2010-01-28,2004-01-02,0.051454,SUN MICROSYSTEMS INC,86681020,86681010,11,3,3571,SUNW,SUNW,012136,01
1,10104,1989-08-03,2024-12-31,2004-01-02,-0.006803,ORACLE CORP,68389X10,68389X10,11,3,7379,ORCL,ORCL,012142,01
2,10107,1994-06-07,2024-12-31,2004-01-02,0.002923,MICROSOFT CORP,59491810,59491810,11,3,7370,MSFT,MSFT,012141,01
3,10108,2002-07-22,2005-08-11,2004-01-02,-0.002887,SUNGARD DATA SYSTEMS INC,86736310,86736310,11,1,7374,SDS,SDS,012144,01
5,10137,2000-12-11,2011-02-25,2004-01-02,-0.011756,ALLEGHENY ENERGY INC,01736110,01736110,11,1,4911,AYE,AYE,001279,01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2692417,89533,2008-11-14,2024-12-31,2020-12-31,-0.005026,WYNN RESORTS LTD,98313410,98313410,11,3,7990,WYNN,WYNN,149318,01
2692418,89617,2015-03-23,2024-12-31,2020-12-31,0.019674,EQUINIX INC,29444U70,29444U70,18,3,4813,EQIX,EQIX,138743,01
2692419,89626,2006-08-11,2024-12-31,2020-12-31,0.022753,C M E GROUP INC,12572Q10,12572Q10,11,3,6200,CME,CME,149070,01
2692420,89641,2012-07-02,2024-12-31,2020-12-31,0.025743,SEAGATE TECHNOLOGY PLC,G7997R10,G7945M10,12,3,3572,STX,STX,150937,01


## TAQ

### Example TAQ pull

In [22]:
query = f"""
    select date, time_m, sym_root, sym_suffix, best_bid, best_ask
    FROM taqmsec.complete_nbbo_20160212
    where sym_root = 'CMCS' 
"""
data = db.raw_sql(query, date_cols=['date'])
print(data.dtypes)
data

date          datetime64[ns]
time_m                object
sym_root              object
sym_suffix            object
best_bid             float64
best_ask             float64
dtype: object


,date,time_m,sym_root,sym_suffix,best_bid,best_ask
0,2016-02-12,04:00:00.013069,CMCS,A,55.79,57.57
1,2016-02-12,04:12:08.372233,CMCS,A,55.79,58.50
2,2016-02-12,04:12:08.372287,CMCS,A,54.86,58.50
3,2016-02-12,04:12:08.372488,CMCS,A,54.86,NaN
4,2016-02-12,04:12:08.372767,CMCS,A,42.00,NaN
...,...,...,...,...,...,...
493104,2016-02-12,18:28:38.456555,CMCS,A,52.66,56.99
493105,2016-02-12,18:40:51.205598,CMCS,A,52.66,57.00
493106,2016-02-12,18:42:39.536882,CMCS,A,52.66,56.99
493107,2016-02-12,18:58:22.056603,CMCS,A,52.66,57.00


In [14]:
import datetime
data = data[(data.time_m >= datetime.time(hour=9, minute=30)) & (data.time_m < datetime.time(hour = 16))]
data['price'] = (data.best_bid+data.best_ask)/2
data['datetime'] = data.apply(lambda x: datetime.datetime.combine(x.date,  x.time_m), axis = 1)

data

/var/folders/yl/b9sf0cwx5vl936gg4zzr3bym0000gn/T/ipykernel_7340/971165293.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['price'] = (data.best_bid+data.best_ask)/2
/var/folders/yl/b9sf0cwx5vl936gg4zzr3bym0000gn/T/ipykernel_7340/971165293.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['datetime'] = data.apply(lambda x: datetime.datetime.combine(x.date,  x.time_m), axis = 1)


,date,time_m,sym_root,sym_suffix,best_bid,best_ask,price,datetime
2514,2016-02-12,09:30:00.000021,FB,None,103.74,103.80,103.770,2016-02-12 09:30:00.000021
2515,2016-02-12,09:30:00.002226,FB,None,103.74,103.80,103.770,2016-02-12 09:30:00.002226
2516,2016-02-12,09:30:00.002333,FB,None,103.73,103.80,103.765,2016-02-12 09:30:00.002333
2517,2016-02-12,09:30:00.002732,FB,None,103.73,103.79,103.760,2016-02-12 09:30:00.002732
2518,2016-02-12,09:30:00.002952,FB,None,103.73,103.79,103.760,2016-02-12 09:30:00.002952
...,...,...,...,...,...,...,...,...
293818,2016-02-12,15:59:59.808476,FB,None,101.98,102.00,101.990,2016-02-12 15:59:59.808476
293819,2016-02-12,15:59:59.808493,FB,None,101.98,102.00,101.990,2016-02-12 15:59:59.808493
293820,2016-02-12,15:59:59.909230,FB,None,101.98,102.00,101.990,2016-02-12 15:59:59.909230
293821,2016-02-12,15:59:59.920454,FB,None,101.98,102.00,101.990,2016-02-12 15:59:59.920454


In [15]:
data_int = data[(data.time_m >= datetime.time(hour=9, minute=30)) & (data.time_m < datetime.time(hour = 16))]
data_int

,date,time_m,sym_root,sym_suffix,best_bid,best_ask,price,datetime
2514,2016-02-12,09:30:00.000021,FB,None,103.74,103.80,103.770,2016-02-12 09:30:00.000021
2515,2016-02-12,09:30:00.002226,FB,None,103.74,103.80,103.770,2016-02-12 09:30:00.002226
2516,2016-02-12,09:30:00.002333,FB,None,103.73,103.80,103.765,2016-02-12 09:30:00.002333
2517,2016-02-12,09:30:00.002732,FB,None,103.73,103.79,103.760,2016-02-12 09:30:00.002732
2518,2016-02-12,09:30:00.002952,FB,None,103.73,103.79,103.760,2016-02-12 09:30:00.002952
...,...,...,...,...,...,...,...,...
293818,2016-02-12,15:59:59.808476,FB,None,101.98,102.00,101.990,2016-02-12 15:59:59.808476
293819,2016-02-12,15:59:59.808493,FB,None,101.98,102.00,101.990,2016-02-12 15:59:59.808493
293820,2016-02-12,15:59:59.909230,FB,None,101.98,102.00,101.990,2016-02-12 15:59:59.909230
293821,2016-02-12,15:59:59.920454,FB,None,101.98,102.00,101.990,2016-02-12 15:59:59.920454


In [16]:
data_int.set_index('datetime', inplace=True)

resampled = data_int.resample('250ms').bfill()
resampled

,date,time_m,sym_root,sym_suffix,best_bid,best_ask,price
datetime,,,,,,,
2016-02-12 09:30:00.000,2016-02-12,09:30:00.000021,FB,None,103.74,103.80,103.770
2016-02-12 09:30:00.250,2016-02-12,09:30:00.252556,FB,None,103.78,103.80,103.790
2016-02-12 09:30:00.500,2016-02-12,09:30:00.627842,FB,None,103.74,103.76,103.750
2016-02-12 09:30:00.750,2016-02-12,09:30:00.757493,FB,None,103.67,103.76,103.715
2016-02-12 09:30:01.000,2016-02-12,09:30:01.022225,FB,None,103.68,103.73,103.705
...,...,...,...,...,...,...,...
2016-02-12 15:59:58.750,2016-02-12,15:59:58.826317,FB,None,101.97,101.99,101.980
2016-02-12 15:59:59.000,2016-02-12,15:59:59.010792,FB,None,101.97,101.99,101.980
2016-02-12 15:59:59.250,2016-02-12,15:59:59.265202,FB,None,101.98,102.00,101.990


### Pulling TAQ data for a set of tickers

In [ ]:
params = {'tickers': tuple(sp500.ticker.values[:3])}
query = """
    SELECT date, time_m, sym_root, sym_suffix, best_bed, best_ask
    FROM taqmsec.complete_nbbo_20160212
    where sym_root IN %(tickers)s
"""
data = db.raw_sql(query, params=params)
data.head()

### TAQ Linker

In [17]:
link = db.raw_sql("""
select sym_root, date, permno, ticker from wrdsapps_link_crsp_taqm.taqmclink as b where b.date = '02/12/2016'""")
link

,sym_root,date,permno,ticker
0,A,2016-02-12,87432,A
1,AA,2016-02-12,24643,AA
2,AAOI,2016-02-12,14145,AAOI
3,AAON,2016-02-12,76868,AAON
4,AAP,2016-02-12,89217,AAP
...,...,...,...,...
5030,VIA,2016-02-12,91066,VIA
5031,WDAY,2016-02-12,13628,WDAY
5032,WK,2016-02-12,15119,WK
5033,WTS,2016-02-12,10606,WTS


In [27]:
sp100 = pd.read_csv('sp100_historical.csv', index_col = 0)
print(sp100.shape)
sp100.head()

(155, 6)


,Code,Name,StartDate,EndDate,IsActiveNow,IsDelisted
0,AAPL,Apple Inc,NaN,NaN,1,0
1,ABBV,AbbVie Inc,2013-01-02,NaN,1,0
2,ABT,Abbott Laboratories,NaN,NaN,1,0
3,ACN,Accenture plc,NaN,NaN,1,0
4,ADBE,Adobe Systems Incorporated,2019-03-20,NaN,1,0


In [28]:
sp100.sort_values(by = 'StartDate')

,Code,Name,StartDate,EndDate,IsActiveNow,IsDelisted
1,ABBV,AbbVie Inc,2013-01-02,NaN,1,0
70,GM,General Motors Company,2013-06-07,NaN,1,0
7,AIG,American International Group Inc,2013-06-07,NaN,1,0
98,META,Meta Platforms Inc.,2013-12-23,NaN,1,0
154,FB,Meta Platforms Inc.,2013-12-23,NaN,1,0
...,...,...,...,...,...,...
148,WBA,Walgreens Boots Alliance Inc,NaN,2023-09-18,0,0
150,WFC,Wells Fargo & Company,NaN,NaN,1,0
151,WMB,Williams Companies Inc,NaN,2013-12-23,0,0
152,WMT,Walmart Inc,NaN,NaN,1,0
